# SLCAN demo on PYNQ


## Introduction

This description shows how to provide SocketCAN with a SLCAN based interface type and automate it.

In order to run this demo there are a whole lot of things to take care:

1. Compile kernel modules **slcan** and **vcan** or install pre-compiled modules.
2. Install **can-utils** package that provides the SocketCAN userspace utilities and tools.
3. Install **python-can** library that provides CAN support for Python.
4. Install **cantools**, a library for parsing DBC files and CAN message encoding/decoding.

An extensive guide for these [prerequisites](#Prerequisites) at the end of this document.

## Required devices

This demo uses the following devices:

* PYNQ-Z1 (with [PYNQ-Z1 v2.3 SDCard image](https://github.com/Xilinx/PYNQ/releases/tag/v2.3))
* [Zubax Babel](https://zubax.com/products/babel) USB-CAN adapter
* CAN slave

## Bringing SLCAN based interface up

Utilize a script to start the SLCAN daemon provided by can-utils and brings `can0` interface up:


In [1]:
import subprocess
folder_dir = %pwd # otherwise os.getcwd()
subprocess.run([folder_dir + "/zubax-babel-can-up.sh"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

CompletedProcess(args=['/home/xilinx/jupyter_notebooks/pynq-mods/slcan_demo/zubax-babel-can-up.sh'], returncode=0, stdout=b'Bringing Zubax Babel CAN interface up\nDone\n', stderr=b'')

### How the script `zubax-babel-can-up.sh` works

SocketCAN provides a SLCAN based interface type. "At first you'll need a special daemon (slcand from can-utils), that will link this serial interface with a virtual CAN device. By default these devices get slcan name base." [[1](https://elinux.org/Bringing_CAN_interface_up)] Another good description about SLCAN is given by [[3](https://uavcan.org/Implementations/Pyuavcan/Tutorials/1._Setup/#can-hardware-backends)].


To use the Zubax Babel USB-to-CAN adapter with SocketCAN you need to run **slcand**:

`sudo slcand -o -c -s6 /dev/serial/by-id/usb-Zubax_Robotics_Zubax_Babel_* can0`

This command creates a new device called `can0` that is connected at `/dev/ttyACM0`. To get a specific device we use the persistent symlink under `/dev/serial/by-id/`. It will open the device when starting (`-o`), close the device when finished (`-c`), and set the speed mode to 6 (`-s6`) which corresponds to a CAN bitrate of 500 Kbit/s.

To enable the interface we run:
`sudo ifconfig can0 up`

It is possible to test the just enabled `can0` interface with a number of utilities. For example **candump** displays messages on the bus in realtime:
`candump can0`

Also note that the kernel module is loaded automatically when the SLCAN daemon starts. This can be seen by executing `lsmod` which now shows
```
Module                  Size  Used by
slcan                  16384  1
```

## Reading messages from CAN bus

To read messages from the CAN bus we create a bus instance and then iterate over received messages. It is also possible to use `recv()` for reading

In [2]:
import can

# Create a bus instance
bus = can.Bus(interface='socketcan', channel='can0')

num_messages = 0

# Iterate over received messages
for msg in bus:
    print(msg)
    num_messages = num_messages + 1
    if num_messages == 10:
        break

Timestamp: 1549021537.136134        ID: 0500    S                DLC:  8    01 2e 00 07 21 2a 01 0d     Channel: can0
Timestamp: 1549021537.139314        ID: 0501    S                DLC:  4    00 00 00 00                 Channel: can0
Timestamp: 1549021537.160877        ID: 0500    S                DLC:  8    01 2e 00 07 21 2a 01 0d     Channel: can0
Timestamp: 1549021537.161866        ID: 0501    S                DLC:  4    00 00 00 00                 Channel: can0
Timestamp: 1549021537.185872        ID: 0500    S                DLC:  8    01 27 00 07 21 2a 01 0d     Channel: can0
Timestamp: 1549021537.186860        ID: 0501    S                DLC:  4    00 00 00 00                 Channel: can0
Timestamp: 1549021537.210881        ID: 0500    S                DLC:  8    01 27 00 07 21 2a 01 0d     Channel: can0
Timestamp: 1549021537.211858        ID: 0501    S                DLC:  4    00 00 00 00                 Channel: can0
Timestamp: 1549021537.235880        ID: 0500    S       

## Decode CAN messages

In [ ]:
import cantools
from pprint import pprint
db = cantools.database.load_file('select_your_device.dbc')
db.messages
example_message = db.get_message_by_name('Your_Sensor_Message')
pprint(example_message.signals)

In [ ]:
num_messages = 0

for msg in bus:
    print(db.decode_message(msg.arbitration_id, msg.data))
    num_messages = num_messages + 1
    if num_messages == 10:
        break

## Prerequisites

As already mentionend SLCAN does not work out of the box on the PYNQ-Z1 board. This guide describes how to configure the board in order to receive messages on the CAN bus via SLCAN based interface types.

### 1) Compile CAN kernel modules

Before using SLCAN compile the kernel modules `slcan` and `vcan`. Do not mistake the userspace library can-utils with the kernel modules needed in order to utilize the library.

Luckily Xilinx already has done some work for us. Following [this guide](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842481/Build+kernel#Buildkernel-TaskDescription) from Xilinx is a good starting point for building kernel modules as it arleady gives some hints of how to configure the Linux kernel for a Zynq AP SoC (which is used on the PYNQ-Z1).

As the kernel source is located on the PYNQ board it is possible to compile it on the target itself at the top-level source directory:
```
sudo -i
cd /usr/src/kernel
```

We invoke the command `make menuconfig`, which is a menu-driven user interface, to choose the additional features `slcan` and `vcan` we want to compile.

```
make ARCH=arm xilinx_zynq_defconfig
make ARCH=arm menuconfig
```

As out CAN device is some kind of networking device we therefore select
`Networking support ---> CAN bus subsystem support ---> CAN Device Drivers ---> Serial / USB serial CAN Adaptors (slcan) ` and
`Virtual Local CAN Interface (vcan)`

We do not want to produce the whole kernel image but only the kernel modules:

`make ARCH=arm UIMAGE_LOADADDR=0x8000 modules`

The `build-essential` package should be already installed (because they are needed for compiling microblaze code from within PYNQ. But you probably must install `apt-get install bc` (bc arbitrary precision calculator language).

The freshly compiled drivers can be found at `cd drivers/net/can/`.
But compiling all modules take a long time. It is better to compile only the two additional kernel modules. At the top-level source directory execute:

`make ARCH=arm UIMAGE_LOADADDR=0x8000 SUBDIRS=drivers/net/can modules`

Instead of installing them we copy the `can` folder to `/lib/modules/4.14.0-xilinx-v2018.2/kernel/drivers/net/can` by executing

`cp -r drivers/net/can /lib/modules/4.14.0-xilinx-v2018.2/kernel/drivers/net/`

### 2) Install CAN related packages

* **can-utils** (package provides the SocketCAN userspace utilities and tools)

`apt-get install can-utils`

* **python-can** (library provides CAN support for Pyhton, e.g. socketcan)

`sudo pip3 install python-can`

* **cantools** (library for parsing DBC files and CAN message encoding/decoding)

`sudo pip3 install cantools (to use .dbc files)`

Remember to **install Python packages as root**. Otherwise they are installed on a different (local) path and Jupyter is not able to locate the package.

### 3) Configure USB device access permissions for Zubax Babel

Following the [Zubax guide](https://kb.zubax.com/pages/viewpage.action?pageId=2195511) we have to

> * Configure udev so that it assigns the correct permissions to attached devices automatically.
> * Add the current user to the group **dialout**, so that the system would enable it to access all serial port devices (including the virtual serial ports) by default.

```
echo 'SUBSYSTEMS=="usb", ATTRS{idVendor}=="1d50", ATTRS{idProduct}=="60c7", MODE="0666"' | sudo tee /etc/udev/rules.d/10-zubax.rules
udevadm control --reload-rules && udevadm trigger
usermod -a -G dialout xilinx
```

### 4) Allow user xilinx to execute shell script as root

To allow the user *xilinx* to execute **only a specific** shell script change /etc/suoders (`sudo visudo`) by adding the following at the end of the file:
`xilinx ALL=(ALL) NOPASSWD: /home/xilinx/jupyter_notebooks/pynq-mods/slcan_demo/zubax-babel-can-up.sh`

Hint: It is better to use a own file and add it as a include at sudoers.d/filename